In [1]:
# https://www.kaggle.com/code/mnassrib/titanic-logistic-regression-with-python/notebook

In [2]:
import pandas as pd
import seaborn as sns
import sklearn
import numpy as np

In [3]:
from sklearn.model_selection import train_test_split

# Import and prepare data for Random Forest

In [4]:
test, train = pd.read_csv("./test.csv"), pd.read_csv("./train.csv")

In [5]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [7]:
train_id, test_id = train["PassengerId"], test["PassengerId"]
train.drop("PassengerId",axis=1,inplace=True), test.drop("PassengerId",axis=1,inplace=True)

(None, None)

In [8]:
target = train["Survived"]
train.drop("Survived",axis=1,inplace=True)

In [9]:
train.shape, test.shape

((891, 10), (418, 10))

In [10]:
train.drop(["Name","Cabin"],axis=1,inplace=True), test.drop(["Name","Cabin"],axis=1,inplace=True)

(None, None)

In [11]:
train["Sex"] = (train["Sex"] == "male").astype("float")
test["Sex"] = (test["Sex"] == "male").astype("float")

In [12]:
train.drop("Ticket",axis=1,inplace=True), test.drop("Ticket",axis=1,inplace=True)

(None, None)

In [13]:
train["Embarked_Q"], train["Embarked_C"], train["Embarked_S"] = ((train["Embarked"] == x).astype("float") for x in ["Q", "C", "S"])
test["Embarked_Q"], test["Embarked_C"], test["Embarked_S"] = ((test["Embarked"] == x).astype("float") for x in ["Q", "C", "S"])

In [14]:
train.drop("Embarked",axis=1,inplace=True), test.drop("Embarked",axis=1,inplace=True)

(None, None)

In [15]:
train["Age"][train["Age"].isnull()] = train["Age"][train["Age"].notnull()].median()
test["Age"][test["Age"].isnull()] = test["Age"][test["Age"].notnull()].median()
test["Fare"][test["Fare"].isnull()] = test["Fare"][test["Fare"].notnull()].median()

/var/folders/k4/dnx7k_tx471_2_1qhs587stm0000gn/T/ipykernel_1958/4159218897.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["Age"][train["Age"].isnull()] = train["Age"][train["Age"].notnull()].median()
/var/folders/k4/dnx7k_tx471_2_1qhs587stm0000gn/T/ipykernel_1958/4159218897.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["Age"][test["Age"].isnull()] = test["Age"][test["Age"].notnull()].median()
/var/folders/k4/dnx7k_tx471_2_1qhs587stm0000gn/T/ipykernel_1958/4159218897.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentatio

# Pure RF method

In [16]:
train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_Q,Embarked_C,Embarked_S
0,3,1.0,22.0,1,0,7.2500,0.0,0.0,1.0
1,1,0.0,38.0,1,0,71.2833,0.0,1.0,0.0
2,3,0.0,26.0,0,0,7.9250,0.0,0.0,1.0
3,1,0.0,35.0,1,0,53.1000,0.0,0.0,1.0
4,3,1.0,35.0,0,0,8.0500,0.0,0.0,1.0


In [17]:
test.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_Q,Embarked_C,Embarked_S
0,3,1.0,34.5,0,0,7.8292,1.0,0.0,0.0
1,3,0.0,47.0,1,0,7.0000,0.0,0.0,1.0
2,2,1.0,62.0,0,0,9.6875,1.0,0.0,0.0
3,3,1.0,27.0,0,0,8.6625,0.0,0.0,1.0
4,3,0.0,22.0,1,1,12.2875,0.0,0.0,1.0


In [18]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [38]:
param_test = {
    "n_estimators":range(1,101,10),
    "oob_score":[True]
}

gsearch = GridSearchCV(
    estimator=RandomForestClassifier(),
    param_grid=param_test,
    scoring='roc_auc',
    cv=10
)

In [39]:
gsearch.fit(train,target)

/opt/homebrew/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:586: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(
/opt/homebrew/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:586: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(
/opt/homebrew/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:586: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(
/opt/homebrew/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:586: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(
/opt/homebrew/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:586: UserWarning: Some inputs do not have OOB scores. This probably me

GridSearchCV(cv=10, estimator=RandomForestClassifier(),
             param_grid={'n_estimators': range(1, 101, 10),
                         'oob_score': [True]},
             scoring='roc_auc')

In [40]:
gsearch.best_score_, gsearch.best_params_

(0.8555202444614209, {'n_estimators': 91, 'oob_score': True})

In [41]:
res = gsearch.best_estimator_.predict(test)
out_dict = {"PassengerId":test_id,"Survived":res}
pd.DataFrame(out_dict).to_csv("submission.csv",index=False)

# RF + Logistic Regression

In [42]:
from logictic import get_model_and_input

In [43]:
lo_m, lo_train, lo_test  = get_model_and_input()

/Users/wujizhu/Desktop/Study/DS/Course 3 Decision Tree/Project Titanic/titanic/logictic.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.Age[train.Age.isnull()] = train.Age[train.Age.notnull()].median()
/Users/wujizhu/Desktop/Study/DS/Course 3 Decision Tree/Project Titanic/titanic/logictic.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.Age[test.Age.isnull()] = test.Age[test.Age.notnull()].median()
/Users/wujizhu/Desktop/Study/DS/Course 3 Decision Tree/Project Titanic/titanic/logictic.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats i

In [44]:
lo_out_train, lo_out_test = lo_m.predict_proba(lo_train)[:,0] ,lo_m.predict_proba(lo_test)[:,0]
rf_out_train, rf_out_test = gsearch.best_estimator_.predict_proba(train)[:,0], gsearch.best_estimator_.predict_proba(test)[:,0]

In [57]:
df_test = pd.DataFrame({"lo":lo_out_test, "rf":rf_out_test})
df_train = pd.DataFrame({"lo":lo_out_train, "rf":rf_out_train})

In [58]:
from sklearn.linear_model import LogisticRegression
merge_model = LogisticRegression()
merge_model.fit(df_train, target)

LogisticRegression()

In [60]:
res = merge_model.predict(df_test)
out_dict = {"PassengerId":test_id,"Survived":res}
pd.DataFrame(out_dict).to_csv("submission.csv",index=False)